In [79]:
import re
import copy
import numpy as np
import pandas as pd
node = []
allDatas = []
siganlList = []
SignalsName = []
messageName = []

In [80]:
def readFile(file_name):
    global node,allDatas,siganlList,SignalsName,messageName
    ''' 得到dbc文件的绝对路径'''
    filePath = file_name
    if filePath:
        print(filePath)
        f = open(filePath, "r")  # 设置文件对象
    else:
        print("读取文件失败！")
        return 0
    """
    NodesPattern:节点
    MessagePattern：消息
    SignalPattern：信号
    """
    NodesPattern = re.compile(r"BU_: (.*)", re.S)
    MessagePattern = re.compile(r"BO_ (.*?) (.*?): (.*?) (.*)", re.S)
#     SignalPattern = re.compile('''SG_ (.*?) : (.*?)\|(.*?)@.*? \((.*?),(.*?)\) \[(.*?)\|(.*?)\] "(.*?)" (.*)''', re.S)
    SignalPattern = re.compile('''SG_ (.*?) : (.*?)\|(.*?)@([0-9])([+|-]) \((.*?),(.*?)\) \[(.*?)\|(.*?)\] "(.*?)" (.*)''', re.S)
    DefaultValue = '''BA_ "GenSig(.*?)" SG_ (\d+) signalname (\d+);'''

    line = f.readline()
    allDatas=[]
    while line:
        """ 匹配出节点 """
        NodesSearched = re.search(NodesPattern, line.strip())
        if NodesSearched:
            node = NodesSearched.group(1).split(" ")
            #print(node)
        """ 匹配出消息 """
        MessageSearched = re.search(MessagePattern, line.strip())
        if MessageSearched:
            siganlList.clear()
            """如果匹配到了message，则获取到message的相关参数 
             比如匹配到了NM_Message_ESC_409，则会解析出改message的一些参数构成list对象['1033', 'NM_Message_ESC_409', '8', 'ESC']
             这四个参数分别是 messgage ID ;message name ; messgae dataLenth ,message sender
             而且把这个list对象 加在了 siganlList 索引0的位置
            """
            Message = list(MessageSearched.groups())
            siganlList.append(Message)
            """ 只 要 message的名字 messageName 列表中"""
            messageName.append(Message[1])
            """读取下一行"""
            line = f.readline()
            """因为有些message并没有定义signal，所以 下一行还是message"""
            MessageSearched = re.search(MessagePattern, line.strip())
            SignalSearched = re.search(SignalPattern, line.strip())
            """下一行如果不是message的内容 就一定是signal的内容了"""
            if not MessageSearched:
                while SignalSearched:
                    """获取信号的参数追加到siganlList"""
                    signal = list(SignalSearched.groups())
                    siganlList.append(signal)
                    """只获取 signal name"""
                    SignalsName.append(signal[0])

                    # 再次解析信号，直到这个message下的信号全部解析完毕
                    line = f.readline()
                    SignalSearched = re.search(SignalPattern, line.strip())
           # print(siganlList)
            c = copy.deepcopy(siganlList)
            allDatas.append(c)
        else:
            line = f.readline()
            MessageSearched = re.search(MessagePattern, line.strip())
    f.close()  # 将文件关闭
    return 1

In [81]:
import os
file_names = []
# path = os.path.abspath('./My_DBC')
# path = os.path.abspath('./opendbc-master/opendbc-master')
path = os.path.abspath('./opendbc-master/opendbc-master-delFalseData')
dirs = os.listdir(path)                    # 获取指定路径下的文件
for i in dirs:
    if os.path.splitext(i)[1] == ".dbc":
        file_names.append(os.path.join(path,i))
file_names

['D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\opendbc-master\\opendbc-master-delFalseData\\acura_ilx_2016_can_generated.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\opendbc-master\\opendbc-master-delFalseData\\acura_ilx_2016_nidec.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\opendbc-master\\opendbc-master-delFalseData\\acura_rdx_2018_can_generated.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\opendbc-master\\opendbc-master-delFalseData\\acura_rdx_2020_can_generated.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\opendbc-master\\opendbc-master-delFalseData\\bmw_e9x_e8x.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\Code\\MyTask\\opendbc-master\\opendbc-master-delFalseData\\cadillac_ct6_chassis.dbc',
 'D:\\--storage--\\program\\Code\\jupyter-notebook\\CarNetworkIDS\\

In [82]:
# readFile(file_names[0])
# print(node)
# print(len(node))
# print(messageName)
# print(len(messageName))
# print(SignalsName)
# print(len(SignalsName))
# print(allDatas)
# print(len(allDatas))

In [83]:
# len(allDatas), allDatas

In [84]:
intervals = [[1,3],[3,5],[2,3]]
for _, end in intervals:
    print(_,end)

1 3
3 5
2 3


In [85]:
intervals = [[1,3],[2,5],[2,3],[3,4]]
# Sort by start point.
# If two intervals share the same start point
# put the longer one to be the first.
intervals.sort(key = lambda x: (x[0], -x[1]))
count = 0
print(intervals)
prev_end = 0
new_intervals = []
for _, end in intervals:
    print(end, prev_end)
    # if current interval is not covered
    # by the previous one
    if end > prev_end:
        count += 1    
        prev_end = end
    else:
        new_intervals.append([_,end])
        prev_end = prev_end

[[1, 3], [2, 5], [2, 3], [3, 4]]
3 0
5 3
3 5
4 5


In [86]:
new_intervals = [val for val in intervals if val not in new_intervals]
new_intervals

[[1, 3], [2, 5]]

In [87]:
def delFullOverlap(intervals):
#     intervals = [[1,3],[2,5],[2,3],[3,4]]
    # Sort by start point.
    # If two intervals share the same start point
    # put the longer one to be the first.
    intervals.sort(key = lambda x: (x[0], -x[1]))
    count = 0
#     print(intervals)
    prev_end = 0
    new_intervals = []
    for _, end in intervals:
#         print(end, prev_end)
        # if current interval is not covered
        # by the previous one
        if end > prev_end:
            count += 1    
            prev_end = end
        else:
            new_intervals.append([_,end])
            prev_end = prev_end
    return [val for val in intervals if val not in new_intervals]

In [88]:
intervals = [[0, 15], [16, 31], [32, 32], [36, 39], [40, 47]]
prev_end = -1
for _, end in intervals:
    print(_, end, prev_end)
    # if current interval is not covered
    # by the previous one
    if _ > prev_end: 
        prev_end = end
    else:
        print("True")

0 15 -1
16 31 15
32 32 31
36 39 32
40 47 39


In [89]:
def checkHalfOverlap(intervals):
    prev_end = -1
    for _, end in intervals:
#         print(end, prev_end)
        # if current interval is not covered
        # by the previous one
        if _ > prev_end: 
            prev_end = end
        else:
            return True
    return False

In [90]:
def findOverlap():
    overlapped_message = []
    false_message = []
    flag = False
    for message in allDatas:
        series = np.zeros(64)
        boundies = []
        message_id = message[0][0]+"_"+message[0][1]
        del message[0]
        for signal in message:
            start = int(signal[1])
            length = int(signal[2])
            endianness = int(signal[3])
            start_row = int(start / 8)
            start_col = 8 - (start - start_row * 8) - 1
            new_start = start_row * 8 + start_col
            if endianness == 0:
                new_end = new_start + length
                count = 0
                for i in range(new_start, new_end):
                    if(i>=64):
                        print(message_id,start,length,endianness,new_start, new_end)
                        false_message.append(message_id)
                        flag = True
                        break
                boundies.append([new_start, new_end-1])
            if flag == True:
                flag = False
                break
#             if endianness == 1:
#                 new_end = new_start - length + 1
#                 for i in range(new_end, new_start + 1):
#                     if(i>=64):
#                         print(message_id,start,length,endianness,new_start, new_end)
#                         false_message.append(message_id)
#                         flag = True
#                         break
#                 boundies.append([new_end, new_start])
#             if flag == True:
#                 flag = False
#                 break
        print(boundies)
        new_boundies = delFullOverlap(boundies)
        print(new_boundies)
        is_overlap = checkHalfOverlap(new_boundies)
        if is_overlap == True:
            overlapped_message.append(message_id)
        break
    return overlapped_message, false_message

In [91]:
readFile(file_names[49])
findOverlap()

D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\stellantis_dasm.dbc
[[16, 31], [48, 51], [56, 63], [2, 15]]
[[2, 15], [16, 31], [48, 51], [56, 63]]


([], [])

In [92]:
overlap_res = []
false_res = []
for file_name in file_names:
    cur_name = file_name.split("\\")[-1]
    readFile(file_name)
    cur_overlap, cur_false = findOverlap()
    overlap_res.append({cur_name: cur_overlap})
    false_res.append({cur_name: cur_false})

D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\acura_ilx_2016_can_generated.dbc
[[0, 15], [16, 31], [32, 32], [36, 39], [40, 47]]
[[0, 15], [16, 31], [32, 32], [36, 39], [40, 47]]
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\acura_ilx_2016_nidec.dbc
[[0, 7], [8, 15]]
[[0, 7], [8, 15]]
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\acura_rdx_2018_can_generated.dbc
[[0, 15], [16, 31], [32, 32], [36, 39], [40, 47]]
[[0, 15], [16, 31], [32, 32], [36, 39], [40, 47]]
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\acura_rdx_2020_can_generated.dbc
[[0, 15], [16, 31], [32, 39], [58, 59], [60, 63]]
[[0, 15], [16, 31], [32, 39], [58, 59], [60, 63]]
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-maste

[[0, 7], [11, 20], [8, 10], [21, 31], [38, 47], [34, 37], [33, 33], [32, 32], [50, 63], [48, 49]]
[[0, 7], [8, 10], [11, 20], [21, 31], [32, 32], [33, 33], [34, 37], [38, 47], [48, 49], [50, 63]]
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\luxgen_s5_2015.dbc
[]
[]
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\mercedes_benz_e350_2010.dbc
[[48, 51], [56, 63], [20, 31], [3, 3], [4, 15]]
[[3, 3], [4, 15], [20, 31], [48, 51], [56, 63]]
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\nissan_leaf_2018.dbc
[]
[]
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\nissan_x_trail_2017.dbc
[]
[]
D:\--storage--\program\Code\jupyter-notebook\CarNetworkIDS\Code\MyTask\opendbc-master\opendbc-master-delFalseData\stellantis_dasm.dbc
[[16, 31

In [93]:
len(overlap_res), overlap_res

(70,
 [{'acura_ilx_2016_can_generated.dbc': []},
  {'acura_ilx_2016_nidec.dbc': []},
  {'acura_rdx_2018_can_generated.dbc': []},
  {'acura_rdx_2020_can_generated.dbc': []},
  {'bmw_e9x_e8x.dbc': []},
  {'cadillac_ct6_chassis.dbc': []},
  {'cadillac_ct6_object.dbc': []},
  {'cadillac_ct6_powertrain.dbc': []},
  {'chrysler_pacifica_2017_hybrid.dbc': []},
  {'chrysler_pacifica_2017_hybrid_private_fusion.dbc': []},
  {'comma_body.dbc': []},
  {'ESR.dbc': []},
  {'FORD_CADS.dbc': []},
  {'ford_cgea1_2_bodycan_2011.dbc': []},
  {'ford_cgea1_2_ptcan_2011.dbc': []},
  {'ford_fusion_2018_adas.dbc': []},
  {'ford_fusion_2018_pt.dbc': []},
  {'ford_lincoln_base_pt.dbc': []},
  {'gm_global_a_chassis.dbc': []},
  {'gm_global_a_high_voltage_management.dbc': []},
  {'gm_global_a_lowspeed.dbc': []},
  {'gm_global_a_lowspeed_1818125.dbc': []},
  {'gm_global_a_object.dbc': []},
  {'gm_global_a_powertrain_expansion.dbc': []},
  {'gm_global_a_powertrain_generated.dbc': []},
  {'honda_accord_2018_can_gener

In [70]:
false_res

[{'acura_ilx_2016_can_generated.dbc': []},
 {'acura_ilx_2016_nidec.dbc': []},
 {'acura_rdx_2018_can_generated.dbc': []},
 {'acura_rdx_2020_can_generated.dbc': []},
 {'bmw_e9x_e8x.dbc': []},
 {'cadillac_ct6_chassis.dbc': []},
 {'cadillac_ct6_object.dbc': []},
 {'cadillac_ct6_powertrain.dbc': []},
 {'chrysler_pacifica_2017_hybrid.dbc': []},
 {'chrysler_pacifica_2017_hybrid_private_fusion.dbc': []},
 {'comma_body.dbc': []},
 {'ESR.dbc': []},
 {'FORD_CADS.dbc': ['1073741824_VECTOR__INDEPENDENT_SIG_MSG']},
 {'ford_cgea1_2_bodycan_2011.dbc': []},
 {'ford_cgea1_2_ptcan_2011.dbc': []},
 {'ford_fusion_2018_adas.dbc': []},
 {'ford_fusion_2018_pt.dbc': []},
 {'ford_lincoln_base_pt.dbc': []},
 {'gm_global_a_chassis.dbc': []},
 {'gm_global_a_high_voltage_management.dbc': []},
 {'gm_global_a_lowspeed.dbc': []},
 {'gm_global_a_lowspeed_1818125.dbc': []},
 {'gm_global_a_object.dbc': []},
 {'gm_global_a_powertrain_expansion.dbc': []},
 {'gm_global_a_powertrain_generated.dbc': []},
 {'honda_accord_2018_

In [ ]:
def findOverlap_2():
    overlapped_message = []
    overlapped_true_message = []
    false_message = []
    flag = False
    for message in allDatas:
        series = np.zeros(64)
        message_id = message[0][0]+"_"+message[0][1]
        del message[0]
        for signal in message:
            start = int(signal[1])
            length = int(signal[2])
            endianness = int(signal[3])
            start_row = int(start / 8)
            start_col = 8 - (start - start_row * 8) - 1
            new_start = start_row * 8 + start_col
            if endianness == 0:
                new_end = new_start + length
                count = 0
                for i in range(new_start, new_end):
                    if(i>=64):
                        print(message_id,start,length,endianness,new_start, new_end)
                        false_message.append(message_id)
                        flag = True
                        break
                    if series[i] == 1:
                        count = count + 1
                if count > 0 :
                    overlapped_message.append(message_id)
                    flag = True
                if count > 0 & count < length :
                    overlapped_true_message.append(message_id)
                    flag = True
                if flag == True:
                    flag = False
                    break
                series[new_start: new_end] = 1
            if endianness == 1:
                new_end = new_start - length + 1
                count = 0
                for i in range(new_end, new_start + 1):
                    if(i>=64):
                        print(message_id,start,length,endianness,new_start, new_end)
                        false_message.append(message_id)
                        flag = True
                        break
                    if series[i] == 1:
                        count = count + 1
                if count > 0 :
                    overlapped_message.append(message_id)
                    flag = True
                if count > 0 & count < length :
                    overlapped_true_message.append(message_id)
                    flag = True
                if flag == True:
                    flag = False
                    break
                series[new_end: new_start + 1] = 1
    return overlapped_message, overlapped_true_message, false_message